# Demo for contextual enhancements

In [ ]:
references:
    [1] Jorg2015 PloS One
    [2] Remco and Franken 2011 IJCV
    [3] Jorg2015 SSVM
    

#### Load the Stanford HARDI dataset and add Rician noise

In [1]:
import numpy as np
from dipy.data import fetch_stanford_hardi, read_stanford_hardi
from dipy.sims.voxel import add_noise

# read data
fetch_stanford_hardi()
img, gtab = read_stanford_hardi()
data = img.get_data()

# add rician noise
data = add_noise(data, 5, 500 , noise_type='rician')

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\smeester\.dipy\stanford_hardi 


In [2]:
from dipy.reconst.csdeconv import auto_response
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel

# Estimate response function
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)
csd_model = ConstrainedSphericalDeconvModel(gtab, response)

# For illustration purposes we will fit only a small portion of the data.
data_small = data[20:50, 55:85, 38:39] # 5 slices
csd_fit = csd_model.fit(data_small)
csd_shm = csd_fit.shm_coeff

#### Create lookup table and perform convolution

In [ ]:
from dipy.denoise.kernel import EnhancementKernel
from dipy.denoise.convolution5d import convolve_5d

# Create lookup table
D33 = 1.0
D44 = 0.02
t = 1
k = EnhancementKernel(D33, D44, t, force_recompute=True)

# add a visualization of the kernel

# Perform convolution
csd_enh = convolve_5d(csd_shm, k)

#### Visualize the raw and enhanced ODFs

In [ ]:
from dipy.reconst.shm import sh_to_sf
from dipy.viz import fvtk
from dipy.data import get_sphere

# Convert raw and enhanced data to discrete form
sphere = get_sphere('symmetric724')
csd_odf = sh_to_sf(csd_shm, sphere, sh_order=8)
csd_enh_odf = sh_to_sf(csd_enh, sphere, sh_order=8)

# Visualize the results with VTK
ren = fvtk.ren()

fodf_spheres = fvtk.sphere_funcs(csd_odf, sphere, scale=2, norm=False, radial_scale=True)
fvtk.add(ren, fodf_spheres)

fodf_spheres_enh = fvtk.sphere_funcs(csd_enh_odf*0.2, sphere, scale=2, norm=False, radial_scale=True)
fodf_spheres_enh.SetPosition(75, 0, 0)

fvtk.add(ren, fodf_spheres_enh)
fvtk.show(ren, size=(600, 600))
fvtk.record(ren, out_path='enhancements.png', size=(2048, 2048), magnification=2)

In [ ]:
# add a sharpening step
# already available in dipy

#### Fiber tracking

In [21]:
from dipy.data import read_stanford_labels
from dipy.reconst.peaks import peak_directions

hardi_img, gtab, labels_img = read_stanford_labels()
data = hardi_img.get_data()
labels = labels_img.get_data()
affine = hardi_img.get_affine()

white_matter = (labels == 1) | (labels == 2)

Dataset is already in place. If you want to fetch it again please first remove the folder C:\Users\smeester\.dipy\stanford_hardi 
All files already in C:\Users\smeester\.dipy\stanford_hardi.


In [102]:
# Calculate peaks and GFA for our ODF

def gfa(samples):
    """The general fractional anisotropy of a function evaluated
    on the unit sphere"""
    diff = samples - samples.mean(-1)[..., None]
    n = samples.shape[-1]
    numer = n * (diff * diff).sum(-1)
    denom = (n - 1) * (samples * samples).sum(-1)
    return np.sqrt(numer / denom)

from dipy.core.ndindex import ndindex
shape = csd_odf.shape[:-1]
csd_peaks = np.zeros(shape+(5,))
csd_gfa = np.zeros(shape)
for idx in ndindex(shape):
    direction, pk, ind = peak_directions(csd_odf[idx], 
                            sphere, 
                            relative_peak_threshold=.8,
                            min_separation_angle=45)
    ll = [-1,-1,-1,-1,-1]
    ll[0:len(ind)] = ind
    csd_peaks[idx]=ll
    csd_gfa[idx] = gfa(csd_odf[idx])

In [128]:
# Use the peaks and GFA for fiber tracking

from dipy.tracking.eudx import EuDX

eu = EuDX(csd_gfa,
          csd_peaks[..., 0],
          seeds=10000,
          odf_vertices=sphere.vertices,
          a_low=0.2,
          mask=white_matter)

csd_streamlines = [streamline for streamline in eu]

In [129]:
csd_streamlines # no output

[]

In [130]:
csdpeaks = peaks_from_model(model=csd_model,
                            data=data_small,
                            sphere=sphere,
                            relative_peak_threshold=.5,
                            min_separation_angle=25,
                            normalize_peaks=True)

from dipy.tracking.eudx import EuDX

eu = EuDX(csdpeaks.gfa,
          csdpeaks.peak_indices[..., 0],
          seeds=10000,
          odf_vertices=sphere.vertices,
          a_low=0.2)

csd_streamlines = [streamline for streamline in eu]

print csd_streamlines # also no output...

[]
